In [1]:
print("all done")

all done


In [2]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
model=ChatGroq(model="qwen/qwen3-32b")

In [4]:
model.invoke("What is the capital of France?")  # Example usage

AIMessage(content="<think>\nOkay, the user is asking for the capital of France. Let me think... I remember that France is a country in Western Europe. The capital is a major city there. I've heard Paris mentioned a lot in relation to France. Isn't Paris known for the Eiffel Tower and the Louvre? Yeah, that's right. So, the capital must be Paris.\n\nWait, but maybe I should double-check to be sure. Sometimes countries have capitals that aren't their largest cities, but in this case, Paris is both the capital and the largest city in France. I don't think there's any confusion with other cities like Lyon or Marseille. No, Paris is definitely the capital. I think that's correct. Let me see if there's any other possibility. No, I can't recall any other city being the capital. So the answer should be Paris.\n</think>\n\nThe capital of France is **Paris**. It is not only the largest city in France but also a major cultural, historical, and economic center, renowned for landmarks such as the E

In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [10]:
embedding_model=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [48]:
#embedding_model.embed_query("What is the capital of France?")

In [12]:
# Data Ingestion

In [13]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [14]:
import os

In [17]:
file_path = os.path.join(os.getcwd(), "data", "sample.pdf")

In [18]:
loader = PyPDFLoader(file_path)

In [19]:
documents = loader.load()

In [49]:
#documents

In [21]:
len(documents)

77

In [22]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150,
    length_function=len
)

In [25]:
docs = text_splitter.split_documents(documents)

In [26]:
len(docs)

765

In [27]:
from langchain.vectorstores import FAISS

In [30]:
vector_store=FAISS.from_documents(docs, embedding_model)

In [32]:
relevant_doc = vector_store.similarity_search("llam2 finetuning benchmark experiments")

In [33]:
relevant_doc = vector_store.similarity_search("llam2 finetuning benchmark experiments", k=10)

In [50]:
#relevant_doc

In [35]:
retriever = vector_store.as_retriever()

In [36]:
retriever.invoke("llama2 finetuning benchmark experiments")

[Document(id='f1575420-e0e2-45d8-9541-22e7f7549ae9', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'c:\\Users\\sunil\\Documents\\document_portal\\notebook\\data\\sample.pdf', 'total_pages': 77, 'page': 73, 'page_label': '74'}, page_content='Llama 2\n7B 0.28 0.25 0.29 0.50 0.36 0.37 0.21 0.34 0.32 0.50 0.28 0.19 0.26 0.32 0.44 0.51 0.30 0.2513B 0.24 0.25 0.35 0.50 0.41 0.36 0.24 0.39 0.35 0.48 0.31 0.18 0.27 0.34 0.46 0.66 0.35 0.2834B 0.27 0.24 0.33 0.56 0.41 0.36 0.26 0.32 0.36 0.53 0.33 0.07 0.26 0.30 0.45 0.56 0.26 0.3570B 0.31 0.29 0.35 0.51 0.41 0.45 0.27 0.34 0.40 0.52 0.36 0.12 0.28 0.31 0.45 0.65 0.33 0.20\nFine-tuned'),
 Document(id='3a2f7038-3ad5-4810-9e3d-da291b76d5f3',

In [38]:
prompt_template = """
        Answer the question based on the context provided below. 
        If the context does not contain sufficient information, respond with: 
        "I do not have enough information about this."

        Context: {context}

        Question: {question}

        Answer:"""

In [39]:
from langchain.prompts import PromptTemplate

In [40]:
prompt=PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [41]:
from langchain_core.output_parsers import StrOutputParser

In [42]:
parser=StrOutputParser()

In [43]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [46]:
from langchain.schema.runnable import RunnablePassthrough

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [47]:
rag_chain.invoke("tell  me about the llama2 finetuning benchmark experiments?")

'<think>\nOkay, let\'s see. The user is asking about the Llama 2 fine-tuning benchmark experiments. I need to look at the context provided to find relevant information.\n\nLooking at the tables, there are two main sections. The first table (Table 3) compares overall performance on academic benchmarks, and the second part shows "Fine-tuned" results. The context mentions that for the models evaluated, the best score between their framework and public results is chosen. The benchmarks are grouped into categories, and individual results are in Section A.2.2, but that\'s not available here.\n\nIn the "Fine-tuned" section under Llama 2, there are numbers for different model sizes (7B, 13B, 34B, 70B). These numbers might represent performance metrics across various benchmarks. For example, the 70B model has values like 0.31, 0.29, etc. The user wants to know about the fine-tuning experiments, so I should focus on these numbers and the context around them.\n\nThe context also mentions that saf

In [1]:
import logging
import os
from datetime import datetime

In [3]:
logs_dir = os.path.join(os.getcwd(), "logs")

In [4]:
os.makedirs(logs_dir, exist_ok=True)